In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Em 'local_data' deverá passar o caminho do dataset em zip

In [ ]:
!unzip '/content/drive/MyDrive/Classroom/Pós-graduação - Inteligencia Artificial/PNEUMONIA.zip'

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras.utils as image
from keras.utils import load_img, img_to_array
import numpy as np
from keras.preprocessing import image

In [ ]:
#Definição da rede neural
classificador = Sequential()

#Criando a camada de convolucao de 32 filtros, e dimensoes 3x3, altura e largura da imagem
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))

#Acelera o processamento com os dados em escala entre 0 e 1 
classificador.add(BatchNormalization())

#Uma matrix de 4 pixels
classificador.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
# Transforma a matrix em um vetor
classificador.add(Flatten())

# Rede neural densa
classificador.add(Dense(units = 1024, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1024, activation = 'relu'))
classificador.add(Dense(units = 512, activation = 'relu'))
classificador.add(Dense(units = 512, activation = 'relu'))

# Zerando 20% das entradas
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))

# Camada de saida com classificação binaria
classificador.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

In [ ]:
# Gerando imagens aleatorias aproveitando os dados existentes
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)

### Em 'pasta_de_treino' e 'pasta_de_teste' deverá passar o caminho da pasta de treino e de teste da sua pasta contendo o dataset.

In [ ]:
gerador_teste = ImageDataGenerator(rescale = 1./255)

# Definindo a base de dados
base_treinamento = gerador_treinamento.flow_from_directory('/content/chest_xray/train',
                                                           target_size = (64, 64),
                                                           batch_size = 32,
                                                           class_mode = 'binary')


base_teste = gerador_teste.flow_from_directory('/content/chest_xray/test',
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary')

### Em defina o passo para o treino. Recomendável setar o tamanho do dataset de treino e para validation_teps o tamanho do dataset de teste

In [ ]:
# Treinando a rede neural
classificador.fit_generator(base_treinamento, steps_per_epoch = 5216 / 32,
                            epochs = 50, validation_data = base_teste,
                            validation_steps = 624 / 32)

In [ ]:
# Carregando uma imagem para classificação
imagem_teste = load_img('/content/chest_xray/test/NORMAL/IM-0001-0001.jpeg',
                              target_size = (64,64))

imagem_teste = img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

#Realizando a previsão
previsao = classificador.predict(imagem_teste)
base_treinamento.class_indices
previsao

In [ ]:
previsao, base_treinamento.class_indices

In [ ]:
classificador.save('/content/gdrive/MyDrive/Classroom/Pós-graduação - Inteligencia Artificial/pneumonia-classifier/pneumonia.h5')